<a href="https://colab.research.google.com/github/zakirangwala/esrb-wrizard/blob/main/notebooks/exploratory_analysis.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Steam + ESRB Exploratory Analysis

This notebook validates the merged dataset (`data/processed/merged_dataset.csv`) and builds intuition for modeling average playtime using Steam metadata, ESRB maturity signals, Steam achievement engagement, and IGDB descriptors.

### Notebook Goals
- Confirm schema, data types, and missing-value patterns after the data pipeline run.
- Create sanity-check visuals (distributions, correlations, ESRB coverage, engagement vs pricing).
- Surface IGDB descriptor coverage and the most common genres/tags to guide feature engineering.
- Provide ready-to-use feature columns for downstream modeling notebooks.

In [2]:
from __future__ import annotations

import math
from collections import Counter
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

sns.set_theme(style="whitegrid", context="talk")
pd.options.display.float_format = "{:,.2f}".format

DATA_PATH = Path("data/processed/merged_dataset.csv")
RANDOM_STATE = 42

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
NUMERIC_COLS = [
    "price_usd",
    "user_score",
    "reviews_positive",
    "reviews_negative",
    "avg_playtime_forever",
    "median_playtime_forever",
    "score_rank",
    "esrb_match_score",
    "achievement_match_score",
    "achievement_achievement_mentions",
    "achievement_achievement_score_mean",
    "achievement_achievement_score_max",
    "achievement_player_completion_pct_mean",
    "achievement_player_rank_mean",
]

df = pd.read_csv(DATA_PATH, low_memory=False)
raw_shape = df.shape

for col in NUMERIC_COLS + ["release_year"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Helpful derived columns
if "avg_playtime_forever" in df.columns:
    df["avg_playtime_hours"] = df["avg_playtime_forever"] / 60.0
    df["log_playtime_hours"] = np.log1p(df["avg_playtime_hours"])
else:
    df["avg_playtime_hours"] = np.nan
    df["log_playtime_hours"] = np.nan

if "price_usd" in df.columns:
    df["price_per_hour"] = df["price_usd"] / df["avg_playtime_hours"].replace(0, np.nan)
    df["price_per_hour"] = df["price_per_hour"].replace([np.inf, -np.inf], np.nan)

if "esrb_esrb_maturity_rating" in df.columns:
    df["esrb_maturity_rating_clean"] = df["esrb_esrb_maturity_rating"].fillna("Unrated")
else:
    df["esrb_maturity_rating_clean"] = "Unrated"

df["has_igdb"] = df["igdb_name"].notna()
df["primary_genre"] = df["genres"].str.split(",").str[0].str.strip()
df["primary_tag"] = df["tags"].str.split(",").str[0].str.strip()

print(f"Loaded {raw_shape[0]:,} rows × {raw_shape[1]} columns from {DATA_PATH}")
display(df.head(3))

## Data Types & Summary Stats

In [ ]:
df.info()

In [ ]:
numeric_summary = df[NUMERIC_COLS + ["avg_playtime_hours", "price_per_hour"]].describe().T
numeric_summary = numeric_summary.dropna(how="all")
display(numeric_summary[["mean", "std", "min", "25%", "50%", "75%", "max"]])

## Missing-Value Landscape

In [ ]:
missing = (
    df.isna().sum()
      .to_frame(name="missing_count")
      .assign(missing_pct=lambda d: (d["missing_count"] / len(df)) * 100)
      .sort_values("missing_pct", ascending=False)
)
display(missing.head(15))

In [ ]:
top_missing = missing.head(20).reset_index().rename(columns={"index": "column"})
plt.figure(figsize=(10, 8))
sns.barplot(data=top_missing, x="missing_pct", y="column", palette="Reds_r")
plt.title("Columns with Highest Missing Percentage")
plt.xlabel("% Missing")
plt.ylabel("")
plt.tight_layout()
plt.show()

## Engagement Distribution (Avg Playtime)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 6))
valid_playtime = df["avg_playtime_hours"].dropna()

sns.histplot(valid_playtime, bins=50, ax=axes[0], color="#1f77b4")
axes[0].set_title("Average Playtime (hours)")
axes[0].set_xlabel("Hours")

sns.histplot(df["log_playtime_hours"].dropna(), bins=50, ax=axes[1], color="#ff7f0e")
axes[1].set_title("Log(1 + Average Playtime)")
axes[1].set_xlabel("log1p(hours)")

plt.tight_layout()
plt.show()

## Price vs. Engagement

In [ ]:
scatter_cols = ["price_usd", "avg_playtime_hours", "esrb_maturity_rating_clean"]
scatter_df = df[scatter_cols].dropna()
if len(scatter_df) > 6000:
    scatter_df = scatter_df.sample(6000, random_state=RANDOM_STATE)

plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=scatter_df,
    x="price_usd",
    y="avg_playtime_hours",
    hue="esrb_maturity_rating_clean",
    alpha=0.6,
    palette="viridis",
)
plt.title("Price vs. Average Playtime (sample)")
plt.xlabel("Price (USD)")
plt.ylabel("Average Playtime (hours)")
plt.legend(title="ESRB", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.show()

## ESRB Coverage & Impact

In [ ]:
rating_counts = df["esrb_maturity_rating_clean"].value_counts().sort_values(ascending=False)
plt.figure(figsize=(10, 5))
sns.barplot(x=rating_counts.index, y=rating_counts.values, palette="deep")
plt.title("ESRB Rating Distribution")
plt.ylabel("Number of Steam Titles")
plt.xlabel("ESRB Rating")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
box_cols = ["esrb_maturity_rating_clean", "avg_playtime_hours"]
box_df = df[box_cols].dropna()
major_ratings = rating_counts.head(6).index
box_df = box_df[box_df["esrb_maturity_rating_clean"].isin(major_ratings)]
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=box_df,
    x="esrb_maturity_rating_clean",
    y="avg_playtime_hours",
    palette="Set2",
)
plt.yscale("log")
plt.title("Playtime Distribution by ESRB Rating (log scale)")
plt.xlabel("ESRB Rating")
plt.ylabel("Avg Playtime (hours, log scale)")
plt.tight_layout()
plt.show()

## Steam Achievements vs. Playtime

In [ ]:
ach_cols = [
    "achievement_achievement_score_mean",
    "achievement_player_completion_pct_mean",
    "avg_playtime_hours",
]
ach_df = df[ach_cols].dropna()

plt.figure(figsize=(10, 6))
sns.regplot(
    data=ach_df,
    x="achievement_achievement_score_mean",
    y="avg_playtime_hours",
    scatter_kws={"alpha": 0.3},
    line_kws={"color": "#d62728"},
)
plt.yscale("log")
plt.title("Achievement Difficulty vs. Avg Playtime")
plt.xlabel("Mean Achievement Score")
plt.ylabel("Avg Playtime (hours, log scale)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
sns.regplot(
    data=ach_df,
    x="achievement_player_completion_pct_mean",
    y="avg_playtime_hours",
    scatter_kws={"alpha": 0.3},
    line_kws={"color": "#2ca02c"},
)
plt.yscale("log")
plt.title("Achievement Completion % vs. Avg Playtime")
plt.xlabel("Mean Completion Percentage")
plt.ylabel("Avg Playtime (hours, log scale)")
plt.tight_layout()
plt.show()

## Correlation Heatmap (Numeric Features)

In [ ]:
heatmap_cols = [
    "price_usd",
    "user_score",
    "reviews_positive",
    "reviews_negative",
    "avg_playtime_hours",
    "median_playtime_forever",
    "achievement_achievement_score_mean",
    "achievement_player_completion_pct_mean",
    "achievement_player_rank_mean",
    "esrb_match_score",
]
heatmap_df = df[heatmap_cols].dropna(axis=1, how="all")
corr = heatmap_df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Correlation Matrix (Selected Numeric Features)")
plt.tight_layout()
plt.show()

## IGDB Descriptors & Genres Coverage

In [ ]:
def explode_semicolon(series):
    tokens = (
        series.dropna()
        .str.split(";")
        .explode()
        .str.strip()
    )
    tokens = tokens[tokens != ""]
    return tokens

descriptor_counts = explode_semicolon(df["igdb_descriptors"]).value_counts().head(15)
genre_counts = explode_semicolon(df["igdb_genres"]).value_counts().head(15)

fig, axes = plt.subplots(1, 2, figsize=(18, 6))
sns.barplot(
    x=descriptor_counts.values,
    y=descriptor_counts.index,
    ax=axes[0],
    palette="mako",
)
axes[0].set_title("Top IGDB Content Descriptors")
axes[0].set_xlabel("Count")
axes[0].set_ylabel("")

sns.barplot(
    x=genre_counts.values,
    y=genre_counts.index,
    ax=axes[1],
    palette="rocket",
)
axes[1].set_title("Top IGDB Genres")
axes[1].set_xlabel("Count")
axes[1].set_ylabel("")

plt.tight_layout()
plt.show()

## Steam Tags & Genres Snapshot

In [ ]:
tag_tokens = explode_semicolon(df["tags"].str.replace(",", ";"))
tag_counts = tag_tokens.value_counts().head(15)
primary_genre_counts = df["primary_genre"].value_counts().head(15)

fig, axes = plt.subplots(1, 2, figsize=(18, 6))
sns.barplot(x=primary_genre_counts.values, y=primary_genre_counts.index, ax=axes[0], palette="Blues_r")
axes[0].set_title("Top Steam Genres")
axes[0].set_xlabel("Count")
axes[0].set_ylabel("")

sns.barplot(x=tag_counts.values, y=tag_counts.index, ax=axes[1], palette="Greens_r")
axes[1].set_title("Top Steam Tags")
axes[1].set_xlabel("Count")
axes[1].set_ylabel("")
plt.tight_layout()
plt.show()

## Modeling Checklist
- Target: `avg_playtime_forever` / `avg_playtime_hours` (consider log-transform).
- Candidate numeric predictors: pricing, review counts, achievement aggregates, IGDB/Steam counts.
- Categorical signals: `esrb_maturity_rating_clean`, `primary_genre`, `primary_tag`, IGDB descriptors.
- Next notebook can vectorize tags/descriptors, split train/test, and train RandomForest or XGBoost; add SHAP later for explainability.